<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#10.1-K-Means聚类算法" data-toc-modified-id="10.1-K-Means聚类算法-1">10.1 K-Means聚类算法</a></span></li><li><span><a href="#10.3-二分K-Means算法" data-toc-modified-id="10.3-二分K-Means算法-2">10.3 二分K-Means算法</a></span></li></ul></div>

# 利用K-Means聚类算法对未标注数据分组
聚类分析试图将相似的对象归为同一簇，将不相似的对象归为不同簇。相似这一概念取决于所选的相似度算法。

## 10.1 K-Means聚类算法
- 优点：容易实现
- 缺点：可能收敛到局部最小值，在大规模数据上收敛得慢
- 适用数据集：数值型数据

伪代码：

    创建k个起始质心（经常是随机选择）
    当任意一个点的簇分配结果发生变化时
        对数据集中每个数据点
            对每个质心
                计算质心与数据点的距离
            将数据点分配到距离最近的簇
        对每一个簇，计算簇中所有点的均值作为质心
        
1. 收集数据：任意方法
2. 准备数据：需要数值型数据来计算距离，也可以将标称型数据映射为二值型数据再计算距离
3. 分析数据：使用任何方法
4. 训练算法：不适应于无监督学习，即无监督学习没有训练过程
5. 测试算法：应用聚类算法，观察结果，可以使用量化的误差指标如平方和来评价算法结果
6. 使用算法：可以用于所希望的任何应用
  

In [1]:
# K-Means聚类支持函数
def loadDataSet(fileName):
    dataMat = []
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip.split('\t')
        fltLine = mat(float, curLine)
        dataMat.append(fltLine)
    return dataMat

def distEclud(vecA, vecB):
    '''计算两个向量的欧式距离
    '''
    return sqrt(sum(power(vecA - vecB, 2)))

def randCent(dataSet, k):
    n = shape(dataSet)[1]
    centroids = mat(zeros((k,n)))
    for j in range(n):
        minJ = min(dataSet[:,j])
        rangeJ = float(max(dataSet[:,j]) - minJ)
        centroids[:,j] = minJ + rangeJ * random.rand(k, 1)
    return centroids

In [2]:
# K-Means 聚类算法
def kMeans(dataSet, k, distMeas=distEclud, createCent=randCent):
    m = shape(dataSet)[0]
    clusterAssment = mat(zeros((m,2)))
    centroids = createCent(dataSet, k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):
            minDist = inf
            minIndex = -1
            for j in range(k):
                distJI = distMeas(centroids[j,:], dataSet[i,:])
                if distJI < minDist:
                    minDist = jistJI
                    MINiNDEX = j
            if clusterAssment[i,0] != minIndex:
                clusterChanged = True
            clusterAssment[i,:] = minIndex, minDist**2
        print centroids
        for cent in range(k):
            ptsInClust = dataSet[nonzero(clusterAssment[:,0].A==cent)[0]]
            centroids[cent,:] = mean(ptsInClust, axis=0)
    return centroids, clusterAssment

## 10.2 使用后处理提高聚类性能

## 10.3 二分K-Means算法
克服K-Means收敛于局部最优的问题，二分K-Means首先将所有点作为一个簇，然后将这个簇一分为二。之后选择其中一个簇进行继续划分，选择哪一个簇进行划分取决于对其划分时候可以最大程度降低SSE的值。上述基于SSE的划分不断重复，直到用户指定的簇数目为止。伪代码如下：

    将所有点看成一个簇
    当簇数目小于K时
        对于每一个簇
            计算总误差
            在给定的簇上面进行K-Means（k=2）
            计算该簇一分为二之后的总误差
        选择使得误差最小的那个簇进行划分操作

In [7]:
# 而分K-Means聚类算法
def biKmeans(dataSets, k, distMeans=distEclud):
    m = shape(dataSet)[0]
    clusterAssment = mat(zeros((m,2)))
    centroid0 = mean(dataSet, axis=0).tolist()[0]    # 创建一个初始簇
    centList = [centroid0]
    while (len(centList) < k):
        # 尝试更新每一个簇
        for j in range(m):
            lowestSSE = inf
            for i in range(len(centList)):
                ptsInCurrCluster = dataSet[nonzero(clusterAssment[:,0].A==i)[0],:]
                centroidMat, splitClustAss = kMeans(ptsInCurrCluster, 2, distMeas)
                sseSplit = sum(splitClustAss[:,1])
                sseNotSplit = sum(clusterAssment[nonzero(clusterAssment[:,0].A !=i)[0],1])
                print "sseSplit and notSplit:" ,sseSplit, sseNotSplit
                if (sseSplit + sseNotSplit) < lowestSSE:
                    bestCentToSplit = i
                    bestNewCents = centroidMat
                    bestClustAss = splitClusAss.copy()
                    lowestSSE = sseSplit + sseNotSplit
        # 更新簇的分配结果
        bestClustAss[nonzero(bestClustAss[:,0].A == 1)[0],0] = len(centList)
        bestClustAss[nonzero(bestClustAss[:,0].A == 0)[0],0] = len(centList)
        print "the bestCentTosplit is ", bestCentToSplit
        print "the len of bestClustAss ", bestClustAss
        centList[bestCentToSplit] = bestNewCents[0,:]
        centList.append(bestNewCents[1,:])
        clusterAssment[nonzero(clusterAssment[:,0].A == bestCentToSplit)[0],:] = bestClustAss
    return mat(centList), clusterAssment
